# Predicting Car Make with CNNs

## Overview

The purpose of this notebook is self-learning. It's an attempt to predict car manufacturer using a convolutional neural network.

## Setup

TensorFlow 2 will be used to build the neural network, and it will be accessed via the Keras front-end.

In [2]:
from glob import glob
from PIL import Image
from keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from keras import layers
from keras import models
from keras import backend
from keras import optimizers
import pandas as pd
import numpy as np
import os
os.chdir('/home/ubuntu/car-images/')

Using TensorFlow backend.


## Data Prep

The entire data set is made up of 64,000 images of cars from [thecarconnection.com](). It was originally sourced by Nicolas Gervais, and details can be found on his Reddit post: https://www.reddit.com/r/MachineLearning/comments/ek5zwv/p_64000_pictures_of_cars_labeled_by_make_model/.

For now, the data is not cross-validated so it's simply split into training, validating, and testing data sets.

In [3]:
# Load and randomize all image file names
full_image_paths = glob("data/full-images/*.jpg")
full_image_paths = np.random.permutation(full_image_paths)

# Process each image and its label
scaled_images = []
makes = []
for idx, image_path in enumerate(full_image_paths):
        
    # Resize each image to 256x256 and convert it to grayscale
    processed_image = Image.open(image_path).resize((100, 100)).convert('L')
    scaled_images.append(np.array(processed_image))
    
    # Get the make of the car from the file name
    makes.append(image_path.split("/")[2].split("_")[0])
    
    # Print update as this can take a little while
    if idx % round(len(full_image_paths) / 10) == 0:
        print(f"Finished {idx} photos, {round((idx / len(full_image_paths)) * 100)}%")
        
# Rescale images for keras/tensorflow
scaled_images_array = np.array(scaled_images)
scaled_images_array = scaled_images_array.reshape(
    scaled_images_array.shape[0], 
    scaled_images_array.shape[1], 
    scaled_images_array.shape[2],
    1
)

# Get number of distant labels present in data set
distinct_y = len(set(list(makes)))

# Transform labels for keras/tensorflow
encoder = LabelBinarizer()
makes_array = encoder.fit_transform(np.array(makes))

# Create x datasets
train_x = scaled_images_array[:round(0.7 * len(scaled_images_array))]
val_x = scaled_images_array[round(0.7 * len(scaled_images_array)):round(0.9 * len(scaled_images_array))]
holdout_x = scaled_images_array[round(0.9 * len(scaled_images_array)):]

# Create y datasets
train_y = makes_array[:round(0.7 * len(makes_array))]
val_y = makes_array[round(0.7 * len(makes_array)):round(0.9 * len(makes_array))]
holdout_y = makes_array[round(0.9 * len(makes_array)):]

Finished 0 photos, 0%
Finished 6447 photos, 10%
Finished 12894 photos, 20%
Finished 19341 photos, 30%
Finished 25788 photos, 40%
Finished 32235 photos, 50%
Finished 38682 photos, 60%
Finished 45129 photos, 70%
Finished 51576 photos, 80%
Finished 58023 photos, 90%


# Model

In [4]:
# Clear TensorFlow session for iterative development
backend.clear_session()

# Build convolutional layers for vision
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape = (100, 100, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Flatten convolutional output and condense into predictions
model.add(layers.Flatten())
model.add(layers.Dense(200, activation = 'relu'))
model.add(layers.Dense(distinct_y, activation = 'softmax'))

# Compile model
model.compile(loss = 'binary_crossentropy', optimizer = optimizers.RMSprop(lr=1e-4), metrics = ['acc'])

# Train model
model.fit(x = train_x, y = train_y, epochs = 10, batch_size = 200, validation_data = (val_x, val_y))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 45127 samples, validate on 12893 samples
Epoch 1/10
45127/45127 [==============================] - 301s 7ms/step - loss: 0.2196 - acc: 0.9716 - val_loss: 0.1053 - val_acc: 0.9762
Epoch 2/10
45127/45127 [==============================] - 300s 7ms/step - loss: 0.1040 - acc: 0.9762 - val_loss: 0.1030 - val_acc: 0.9762
Epoch 3/10
45127/45127 [==============================] - 300s 7ms/step - loss: 0.1009 - acc: 0.9762 - val_loss: 0.1014 - val_acc: 0.9762
Epoch 4/10
45127/45127 [==============================] - 302s 7ms/step - loss: 0.0973 - acc: 0.9764 - val_loss: 0.0991 - val_acc: 0.9763
Epoch 5/10
45127/45127 [==============================] - 301s 7ms/step - loss: 0.0932 - acc: 0.9767 - val_loss: 0.0982 - val_acc: 0.9764
Epoch 6/10
45127/45127 [==============================] - 302s 7ms/step - loss: 0.0889 - acc: 0.9774 - val_loss: 0.0947 - val_acc: 0.9768
Ep

# Evaluate on Holdout Set

Evaluate on a true holdout set for final performance metric --> about 98% accuracy.

In [5]:
holdout_loss, holdout_acc = model.evaluate(holdout_x, holdout_y)

6447/6447 [==============================] - 14s 2ms/step


In [6]:
holdout_acc

0.97913391573107